In [89]:
import torch
from torch.optim import lr_scheduler
import pandas as pd
import numpy as np

import os, sys
sys.path.insert(0, '/content/Data-Science-Project-2-2021-2-Nowcasting/')

from utils.config import cfg
from utils.blocks.forecaster import Forecaster
from utils.blocks.encoder import Encoder
from utils.tools.ordered_easydict import OrderedDict
from utils.blocks.module import EF
from utils.loss import Weighted_mse_mae
from utils.train_and_test import train_and_test
from experiment.net_params import conv2d_params
from utils.blocks.module import Predictor

if not os.path.exists(cfg.GLOBAL.MODEL_SAVE_DIR):
    os.makedirs(cfg.GLOBAL.MODEL_SAVE_DIR)

## Train-Valid-Test Split

In [90]:
from utils.tools.train_test_split import *

train_test_split(cfg.ONM_PD.FOLDER_ALL, ratio=(0.8,0.05,0.15))

## Experiment Conv2d

In [91]:
batch_size = cfg.GLOBAL.BATCH_SIZE
max_iterations = 10000
test_iteration_interval = 1000
test_and_save_checkpoint_iterations = 1000

LR = 1e-4

criterion = Weighted_mse_mae().to(cfg.GLOBAL.DEVICE)

model = Predictor(conv2d_params).to(cfg.GLOBAL.DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=1e-6)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.7)
folder_name = "conv2d"

### Verifying model network

In [92]:
model

Predictor(
  (model): Sequential(
    (conv1_relu_1): Conv2d(5, 64, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (relu_conv1_relu_1): ReLU(inplace=True)
    (conv2_relu_1): Conv2d(64, 192, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_conv2_relu_1): ReLU(inplace=True)
    (conv3_relu_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (relu_conv3_relu_1): ReLU(inplace=True)
    (deconv1_relu_1): ConvTranspose2d(192, 192, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (relu_deconv1_relu_1): ReLU(inplace=True)
    (deconv2_relu_1): ConvTranspose2d(192, 64, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (relu_deconv2_relu_1): ReLU(inplace=True)
    (deconv3_relu_1): ConvTranspose2d(64, 64, kernel_size=(7, 7), stride=(5, 5), padding=(1, 1))
    (relu_deconv3_relu_1): ReLU(inplace=True)
    (conv3_relu_2): Conv2d(64, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_relu_2): ReLU(inplace=True)
    (conv3_3

### Verifying input/output shape

In [93]:
data = torch.randn(5, 4, 1, 480, 480)
output = model(data.cuda())
print(output.size())

torch.Size([20, 4, 1, 480, 480])


### Train and Test

In [ ]:
train_and_test(model, optimizer, criterion, exp_lr_scheduler, batch_size, max_iterations, test_iteration_interval, test_and_save_checkpoint_iterations, folder_name)

  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
 21%|██▏       | 4293/20000 [1:11:23<2:17:53,  1.90it/s]